[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# UK COVID19 Vaccine Coverage Dashboard

The dashboard at the end of this page shows vaccination coverage percentage for each age group in the UK. 

In [47]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [75]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 200

## Load initial data from disk

An initial data is loaded to setup the plots. Click on the "Retrieve" button to get the updated data.

In [89]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("initialdata.json", "rt") as INFILE:
    jsondata = json.load(INFILE)

## Wrangle the data


In [101]:

def wrangle_data(rawdata):
    """ 
    :param rawdata: dict, data from json file or API call. 
    :return (df): dataframe for plotting
    """
    # Calculate lenghth of nested list to use indexes
    agegroupsize = len(rawdata["data"][0]["vaccinationdemograph"])
    
    # Add data needed into seperate lists
    agegroups = [rawdata["data"][0]["vaccinationdemograph"][i]["age"] for i in range(agegroupsize)]
    twodoses = [rawdata["data"][0]["vaccinationdemograph"][i]["cumVaccinationCompleteCoverageByVaccinationDatePercentage"] for i in range(agegroupsize)]
    booster = [rawdata["data"][0]["vaccinationdemograph"][i]["cumVaccinationThirdInjectionUptakeByVaccinationDatePercentage"] for i in range(agegroupsize)]
    
    # Clean up data for better plot presentation
    lists_preprocessing(agegroups, twodoses, booster)
    
    # Dataframe creation
    df = pd.DataFrame(index=agegroups, columns=['2 Doses', 'Booster'])
    for i in range(len(agegroups)):
        ageband = agegroups[i]
        df.loc[ageband, '2 Doses'] = twodoses[i]
        df.loc[ageband, 'Booster'] = booster[i]
    return df


def lists_preprocessing(l1, l2, l3):
    """
    :param l1, l2, l3: list, contains data to use as index, and column values for datafram
    :return l1, l2, l3: cleaned-up list for dataframe plotting.
    """
    for group in l1:
        if "+" in group:
            if "90" in group:
                break 
            else:
                i = l1.index(group)
                l1.pop(i)
                l2.pop(i)
                l3.pop(i)
    return l1, l2, l3

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in this cell as below:
df=wrangle_data(jsondata) # df is the dataframe for plotting

## Download current data

Click on the blue Retrieve button to retrieve the data of UK vaccination coverage for each group. If data succesfully retrieved, the button will turn green. 

In [92]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    filters = ['areaType=nation', 'areaName=England']
    structure = {'date': 'date','vaccinationdemograph': 'vaccinationsAgeDemographics'}
    api = Cov19API(filters=filters, structure=structure)
    vacdemo=api.get_json()

    with open ("vacdemo", "wt") as OUTF:
        json.dump(vacdemo, OUTF)
    return vacdemo # return data read from the API

In [119]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ 
    Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. 
    """
    global df
    try:
        # Get fresh data from the API. 
        apidata=access_api()

        # Wrangle the data and overwrite the dataframe for plotting
        df=wrangle_data(apidata)

        refresh_graph()
        
        # Update button to indicate successful retrieval
        apibutton.icon="check"
        apibutton.button_style = 'success'
        apibutton.description = 'Retrieved'
    
    except Exception as e:
        # Handle all kinds of errors
        apibutton.icon="times"
        apibutton.button_style = 'danger'
        apibutton.description = "Retry"
    
    
    
apibutton=wdg.Button(
    description='RETRIEVE DATA', 
    disabled=False,
    button_style='primary', 
    tooltip="Click on it!",
    icon='download'
)

apibutton.on_click(api_button_callback) 

display(apibutton)

Button(button_style='primary', description='RETRIEVE DATA', icon='download', style=ButtonStyle(), tooltip='Cli…


## Graphs and Analysis

There are three different plots available for users to choose from. The default plot is a combination of the other two plots:
1. 2 doses coverage percentage for different age groups in UK
2. Booster shot coverage percentage for different age groups in UK
Choose the plot you would like to see in the drop down menu.

In [97]:
def plot_age_df(vaccination):
    # To combine both plots into one plot
    if vaccination == 'Combined':
        df['2 Doses'].plot(label='2 Doses Coverage', color='orange')
        df['Booster'].plot(label='Booster Shots Coverage', color='red')
        plt.legend()
    
    # To assign different colors to different plots
    elif vaccination == '2 Doses':
        color = 'orange' 
        df[vaccination].plot(color=color)
    else:
        color = 'red'
        df[vaccination].plot(color=color)
        
    # Add elements and adjust style to plot
    plt.xlabel('Age Group', fontweight='bold', labelpad=15)
    plt.ylabel('Vaccination Coverage(%)', fontweight='bold', labelpad=15)
    plt.grid(axis='y', linestyle='--', color='grey', alpha=0.5)
    plt.ylim(0,100) # Set y-axis scale to same across plots
    plt.xticks(ticks=np.arange(len(df.index)), labels=df.index, rotation=45, fontsize=8)
    
    plt.show() 
    
# Drop down widget
dosesNo = wdg.Dropdown(
    options = ['Combined', '2 Doses', 'Booster'],
    value = 'Combined',
    description = 'Plots: ',
    disabled = False,
)

def refresh_graph():
    current = dosesNo.value
    if current == dosesNo.options[0]:
        other = dosesNo.options[1]
    else:
        other = dosesNo.options[0]
    dosesNo.value = other # forces the redraw
    dosesNo.value = current # now we can change it back
    
# Connect the plotting function and the widget    
graph = wdg.interactive_output(plot_age_df, {'vaccination': dosesNo})

# Actually displays the graph
display(dosesNo, graph)

Dropdown(description='Plots: ', options=('Combined', '2 Doses', 'Booster'), value='Combined')

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice**  Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).